# 法律助理

测试和实验GPT3.5在中国法律助理场景的应用：对中国法律的理解，对法律条款和陈述的综合分析能力的测试。
尝试利用以前bar靠的题库建立一个测试集，在其基础上对基础模型的能力建立比较准确的认知。后面可以体系性的开发提效。


In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain
from langchain.llms import OpenAI
GPT4='GPT-4-32k'
GPT3_5='text-davinci-003'
llm = OpenAI(model_name=GPT3_5, temperature=0.0, max_tokens=3000)


构建框架：数据

In [2]:

laws = [
"犯罪嫌疑人自被侦查机关第一次讯问或者采取强制措施之日起，有权委托辩护人; 在侦查期间，只能委托律师作为辩护人。被告人有权随时委托辩护人。",
"辩护律师会见在押的犯罪嫌疑人、被告人，可以了解案件有关情况，提供法律咨询等;自案件移送审查起诉之日起，可以向犯罪嫌疑人、被告人核实有关证据。辩护律师会见犯罪嫌疑人、被告人时不被监听。",
"辩护人收集的有关犯罪嫌疑人不在犯罪现场、未达到刑事责任年龄属于依法不负刑事责任的精神病人的证据，应当及时告知公安机关、人民检察院。",
"危害国家安全犯罪、恐怖活动犯罪案件，在侦查期间辩护律师会见在押的犯罪嫌疑人，应当经侦查机关许可",
"在交付执行前，暂予监外执行由交付执行的人民法院决定; 在交付执行后，暂予监外执行由监狱或者看守所提出书面意见，报省级以上监狱管理机关或者设区的市一级以上公安机关批准。",
"不符合暂予监外执行条件的罪犯通过贿赂等非法手段被暂予监外执行的，在监外执行的期间不计人执行刑期。罪犯在暂予监外执行期间脱逃的，脱逃的期间不计人执行刑期。",
"被决定收监执行的社区矫正人员在逃的，社区矫正机构应当立即通知公安机关，由公安机关负责追捕。",
"对被判处管制、宣告缓刑、假释或者暂予监外执行的罪犯，依法实行社区矫正，由社区矫正机构负责执行。"
]

statements = [
"犯罪嫌疑人高某因为绑架罪被立案侦查，自被第一次讯问后或被采取强制措施之日起，高某有权聘请其在医院工作的朋友袁某作其辩护人",
"黄某因涉嫌盗窃罪被逮捕，在侦查阶段，其聘请的辩护律师贾某会见黄某时，可以向黄某核实有关证据",
"石某因涉嫌故意杀人罪被立案侦查，在审查起诉阶段，辩护律师刘某应当将收集的所有证明石某无罪、罪轻的证据及时告知检察院",
"李某因为涉嫌间谍罪被立案侦查，在侦查阶段，辩护律师张某会见在押的李某时，须经侦查机关许可",
"张某因盗窃被判处5年有期徒刑，在交付执 行前，突患严重疾病，需保外就医，应当由省级以上监狱管理机关决定张某暂予监外执行",
"不符合暂予监外执行条件的罪犯通过贿赂等非法手段被暂予监外执行的，在监外执行的期间不计人执行刑期",
"罪犯在暂予监外执行期间脱逃的，脱逃的期间不计人执行刑期。被决定收监执行的社区矫正人员在逃的，由社区矫正机构负责追捕",
"对暂予监外执行的罪犯，由公安机关交所在单位或者基层组织予以考察"
]

最简单的推理测试：找到好的prompt很不容易，需要对法律场景、问题类型、模型与人在这样问题上的思维方式及其偏见（inductive bias）有深刻的理解后（经过很多尝试后）才能生成。

In [3]:
qTemplate="请仔细理解以下法律条款和陈述后回答问题 法律条款：{law} 陈述：{statement} 问题：{question}"

question = "以上法律条款适用于以上陈述的先决条件是什么？这个条件满足了吗？为什么？陈述是正确的吗？为什么？"

qPrompt = PromptTemplate(input_variables=["law","statement", "question"], template=qTemplate)

i=0
law=laws[i]
statement=statements[i]

#llm(qPrompt.format(law=law, statement=statement, question=question))

qChain = LLMChain(llm=llm, verbose=True, prompt=qPrompt)
qChain.run(law=law, statement=statement, question=question)



> Entering new LLMChain chain...
Prompt after formatting:
请仔细理解以下法律条款和陈述后回答问题 法律条款：犯罪嫌疑人自被侦查机关第一次讯问或者采取强制措施之日起，有权委托辩护人; 在侦查期间，只能委托律师作为辩护人。被告人有权随时委托辩护人。 陈述：犯罪嫌疑人高某因为绑架罪被立案侦查，自被第一次讯问后或被采取强制措施之日起，高某有权聘请其在医院工作的朋友袁某作其辩护人 问题：以上法律条款适用于以上陈述的先决条件是什么？这个条件满足了吗？为什么？陈述是正确的吗？为什么？

> Finished chain.


'\n\n先决条件是袁某必须是律师，才能在侦查期间作为辩护人。这个条件不满足，因为袁某是在医院工作的朋友，而不是律师。陈述是不正确的，因为袁某不是律师，不能在侦查期间作为辩护人。'

以下是草稿本：

In [ ]:

vTemplate="根据以下法律条款和场景描述，请验证以下假设是否属实。\
法律条款描述：{law}\
场景描述：{case} \
假设：{assumption}"
assumption = "袁某是律师"
vPrompt = PromptTemplate(input_variables=["law","case", "assumption"], template=vTemplate)
vChain = LLMChain(llm=llm, verbose=True, prompt=vPrompt)
vChain.run(law=law, case=case, assumption=assumption)

answers = []
for i in range(2):
    answers.append (chain.run(law=laws[i], case=cases[i], question=question))
print(answers)



